In [1]:
from PIL import Image
from scipy.fftpack import fft
from sklearn.decomposition import PCA
import numpy as np
import os
import cv2
import pandas as pd

In [2]:
folder="./Tamil-English/AUDIO/"
audio_clips=os.listdir(folder)
pict_Path="./Tamil-English/PIC/"
specgrams=os.listdir(pict_Path)
totalac=len(audio_clips)
print("No. of .wav files in audio folder = ",totalac)
totalsc=len(specgrams)
print("No. of .png files in picture folder = ",totalsc)

No. of .wav files in audio folder =  382
No. of .png files in picture folder =  382


In [3]:
d=pd.read_excel(r"./Tamil-English/D.xlsx",sheet_name='Sheet1')
t=pd.read_excel(r"./Tamil-English/T.xlsx",sheet_name='Sheet1')

In [4]:
d['File']=d['File'].astype("string")
d['Translation']=d['Translation'].astype("string")
d['File']=d['File'].str.replace(".OGG","")
d['Translation']=d['Translation'].str.lower()
d=d.set_index('File')
d.head()

C:\Users\HP\AppData\Local\Temp/ipykernel_32896/1614673228.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  d['File']=d['File'].str.replace(".OGG","")


,Translation,Transcript
File,,
D1,"hey,i will be late,you all go","DEI, nan vaara late aagum,neenge yelarum ponga"
D2,"guys,there is an emergency,we need blood urgen...","guys,emergency ya blood vennum,inda number ku ..."
D3,"dude,i am not coming ,got some work","machan,nan varla ,yenneku work irukudhu"
D4,"lets eat here,i am really hungry ,have to eat ...","lets eat here da,yenuku pasikidhu,sapte aganum"
D5,"he will come,he has informed already in the gr...","he will not come machan,avan nethe inform pani..."


In [5]:
t['File']=t['File'].astype("string")
t=t.set_index('File')
t.head()

,Translation,Transcript
File,,
T1,Everytime by keeping the book open don't prete...,Everythime book vachta study paanalpla acting ...
T2,Sarva be careful while playing in evening,Sarva evening play paana podu carefulla play p...
T3,Today we cleaned the house and read books,Today house clean panno Today read panno
T4,Play well tomorrow,Tomorrow nalla play paanuo
T5,Prepare well for tomorrow's exam,Tomorrow examke naala prepare pannu


In [6]:
y=pd.concat([d,t],sort=True)

In [7]:
y.head(10)

,Transcript,Translation
File,,
D1,"DEI, nan vaara late aagum,neenge yelarum ponga","hey,i will be late,you all go"
D2,"guys,emergency ya blood vennum,inda number ku ...","guys,there is an emergency,we need blood urgen..."
D3,"machan,nan varla ,yenneku work irukudhu","dude,i am not coming ,got some work"
D4,"lets eat here da,yenuku pasikidhu,sapte aganum","lets eat here,i am really hungry ,have to eat ..."
D5,"he will not come machan,avan nethe inform pani...","he will come,he has informed already in the gr..."
D6,"inndha week poga vendam da,I have got some pla...","we will not go this week man,i got some plans ..."
D7,"its your wish machi,if you ask me I will not r...","its your wish,if you ask me i will not recomme..."
D8,"are we playing this week,yenku thirinju yaarum...",guys are we playing this week? i think no one ...
D9,nan just velliye irundu vanthen,i have just came from outside


In [8]:
X = []
Y = []
for i in range(0,totalsc):
    image = cv2.imread(pict_Path+specgrams[i])
    image = cv2.resize(image,(100,100), interpolation = cv2.INTER_CUBIC)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    X.append(image.T)
    Y.append(y.loc[(folder+specgrams[i]).split('/')[-1][:-4]]['Transcript'])

In [56]:
X2Y = list([X[i],Y[i]] for i in range(len(X)))
df = pd.DataFrame(X2Y, columns = ['X', 'Y'])

In [61]:
from sklearn.model_selection import train_test_split
X_train,y_train, X_test,y_test = train_test_split(X,Y, test_size=0.2)

In [62]:
import keras
import tensorflow as tf
import keras.backend as K
from keras.models import Model
from keras.layers import Input
from keras.layers import TimeDistributed
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Lambda
from keras.layers import Dropout
from keras.regularizers import l2
from keras.initializers import random_normal
from keras.utils.conv_utils import conv_output_length
from keras.layers import GaussianNoise
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers.recurrent import SimpleRNN
from keras.layers import Dense, Activation, Bidirectional, Reshape,Flatten, Lambda, Input,\
    Masking, Convolution1D, BatchNormalization, GRU, Conv1D, RepeatVector, Conv2D
from keras.layers import ZeroPadding1D, Convolution1D, ZeroPadding2D, Convolution2D, MaxPooling2D, GlobalMaxPooling2D
from keras.layers import TimeDistributed, Dropout
from keras.layers.merge import add  # , # concatenate BAD FOR COREML
from keras.utils.conv_utils import conv_output_length
from keras.activations import relu
from keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam, Nadam,SGD

In [65]:
def clipped_relu(x):
    return relu(x, max_value=20)
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
def ctc(y_true, y_pred):
    return y_pred
def ds1_dropout(input_dim=26, fc_size=2048, rnn_size=512, dropout=[0.1, 0.1, 0.1], output_dim=29):
    """ DeepSpeech 1 Implementation with Dropout
    Architecture:
        Input MFCC TIMEx26
        3 Fully Connected using Clipped Relu activation function
        3 Dropout layers between each FC
        1 BiDirectional LSTM
        1 Dropout applied to BLSTM
        1 Dropout applied to FC dense
        1 Fully connected Softmax
    Details:
        - Uses MFCC's rather paper's 80 linear spaced log filterbanks
        - Uses LSTM's rather than SimpleRNN
        - No translation of raw audio by 5ms
        - No stride the RNN
    Reference:
        https://arxiv.org/abs/1412.5567
    """
    from keras.utils.generic_utils import get_custom_objects
    get_custom_objects().update({"clipped_relu": clipped_relu})
    K.set_learning_phase(1)

    # Creates a tensor there are usually 26 MFCC
    input_data = Input(name='the_input', shape=(None, input_dim))  # >>(?, max_batch_seq, 26)

    # First 3 FC layers
    init = random_normal(stddev=0.046875)
    x = TimeDistributed(Dense(fc_size, name='fc1', kernel_initializer=init, bias_initializer=init, activation=clipped_relu))(input_data)  # >>(?, 778, 2048)
    x = TimeDistributed(Dropout(dropout[0]))(x)
    x = TimeDistributed(Dense(fc_size, name='fc2', kernel_initializer=init, bias_initializer=init, activation=clipped_relu))(x)  # >>(?, 778, 2048)
    x = TimeDistributed(Dropout(dropout[0]))(x)
    x = TimeDistributed(Dense(fc_size, name='fc3', kernel_initializer=init, bias_initializer=init, activation=clipped_relu))(x)  # >>(?, 778, 2048)
    x = TimeDistributed(Dropout(dropout[0]))(x)

    # Layer 4 BiDirectional RNN
    x = Bidirectional(LSTM(rnn_size, return_sequences=True, activation=clipped_relu, dropout=dropout[1],
                                kernel_initializer='he_normal', name='birnn'), merge_mode='sum')(x)


    # Layer 5+6 Time Dist Dense Layer & Softmax
    # x = TimeDistributed(Dense(fc_size, activation=clipped_relu, kernel_initializer=init, bias_initializer=init))(x)
    x = TimeDistributed(Dropout(dropout[2]))(x)
    y_pred = TimeDistributed(Dense(output_dim, name="y_pred", kernel_initializer=init, bias_initializer=init, activation="softmax"), name="out")(x)

    # Change shape
    labels = Input(name='the_labels', shape=[None,], dtype='int32')
    input_length = Input(name='input_length', shape=[1], dtype='int32')
    label_length = Input(name='label_length', shape=[1], dtype='int32')

    # Keras doesn't currently support loss funcs with extra parameters
    # so CTC loss is implemented in a lambda layer
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred,
                                                                       labels,
                                                                       input_length,
                                                                       label_length])

    model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)

    return model

In [71]:
# DeepSpeech1 with Dropout
model1 = ds1_dropout(input_dim=305, fc_size=512, rnn_size=512,dropout=[0.1,0.1,0.1], output_dim=29)
print(model1.summary(line_length=80))


Model: "model_4"
________________________________________________________________________________
Layer (type)              Output Shape      Param #  Connected to               
input_4 (InputLayer)      [(None, 100, 100) 0                                   
________________________________________________________________________________
time_distributed_34 (Time (None, 100, 512)  51712    input_4[0][0]              
________________________________________________________________________________
time_distributed_35 (Time (None, 100, 512)  0        time_distributed_34[0][0]  
________________________________________________________________________________
time_distributed_36 (Time (None, 100, 512)  262656   time_distributed_35[0][0]  
________________________________________________________________________________
time_distributed_37 (Time (None, 100, 512)  0        time_distributed_36[0][0]  
________________________________________________________________________________
time_distri

In [72]:
opt = Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-8, clipnorm=5)
model1.compile(optimizer=opt,loss=ctc)

In [73]:
v_split = 0.2
model1.fit_generator(generator=X_train,
                        validation_data=X_test,
                        initial_epoch=0,
                        verbose=1,
                        class_weight=None,
                        workers=1,)

ValueError: in user code:

    c:\users\hp\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    c:\users\hp\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\hp\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\hp\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\hp\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\users\h

In [ ]:
open('model.json', 'w').write(model.to_json())
model.save_weights('weights.h5')
print("Model Saved")

In [ ]:
open('model.json', 'w').write(model.to_json())
model.save_weights('weights.h5')
print("Model Saved")

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.plot(model.history.history['loss'],'red')
plt.plot(model.history.history['acc'],'blue')
plt.plot(model.history.history['val_loss'],'yellow')
plt.plot(model.history.history['val_acc'],'black')
plt.show()